In [200]:
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model, ensemble
from sklearn import cross_validation, metrics
from sklearn.cross_validation import cross_val_score
from sklearn.externals import joblib

import tensorflow as tf

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [201]:
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'ridge_minimal'

In [202]:
train = pd.read_csv('data/train_weird.csv')
test = pd.read_csv('data/test_weird.csv')

In [203]:
train.head()

,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,CentralAir,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Functional,GarageFinish,Fence,YearBuilt,YearRemodAdd,GarageYrBlt,MoSold,YrSold,LowQualFinSF,MiscVal,PoolQC,PoolArea,MSSubClass,MSZoning,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,Foundation,SaleType,SaleCondition,IsRegularLotShape,IsLandLevel,IsLandSlopeGentle,IsElectricalSBrkr,IsGarageDetached,IsPavedDrive,HasShed,Remodeled,RecentRemodel,VeryNewHouse,Has2ndFloor,HasMasVnr,HasWoodDeck,HasOpenPorch,HasEnclosedPorch,Has3SsnPorch,HasScreenPorch,HighSeason,NewerDwelling,Neighborhood_Good,SaleCondition_PriceDown,BoughtOffPlan,BadHeating,TotalArea,TotalArea1st2nd,Age,TimeSinceSold,SeasonSold,YearsSinceRemodel,SimplOverallQual,SimplOverallCond,SimplPoolQC,SimplGarageCond,SimplGarageQual,SimplFireplaceQu,SimplFunctional,SimplKitchenQual,SimplHeatingQC,SimplBsmtFinType1,SimplBsmtFinType2,SimplBsmtCond,SimplBsmtQual,SimplExterCond,SimplExterQual,NeighborhoodBin,_MSSubClass_20,_MSSubClass_30,_MSSubClass_40,_MSSubClass_45,_MSSubClass_50,_MSSubClass_60,_MSSubClass_70,_MSSubClass_75,_MSSubClass_80,_MSSubClass_85,_MSSubClass_90,_MSSubClass_120,_MSSubClass_180,_MSSubClass_190,_MSZoning_FV,_MSZoning_RH,_MSZoning_RL,_MSZoning_RM,_LotConfig_Corner,_LotConfig_CulDSac,_LotConfig_FR2,_LotConfig_FR3,_LotConfig_Inside,_Neighborhood_Blmngtn,_Neighborhood_Blueste,_Neighborhood_BrDale,_Neighborhood_BrkSide,_Neighborhood_ClearCr,_Neighborhood_CollgCr,_Neighborhood_Crawfor,_Neighborhood_Edwards,_Neighborhood_Gilbert,_Neighborhood_IDOTRR,_Neighborhood_MeadowV,_Neighborhood_Mitchel,_Neighborhood_NAmes,_Neighborhood_NPkVill,_Neighborhood_NWAmes,_Neighborhood_NoRidge,_Neighborhood_NridgHt,_Neighborhood_OldTown,_Neighborhood_SWISU,_Neighborhood_Sawyer,_Neighborhood_SawyerW,_Neighborhood_Somerst,_Neighborhood_StoneBr,_Neighborhood_Timber,_Neighborhood_Veenker,_Condition1_Artery,_Condition1_Feedr,_Condition1_Norm,_Condition1_PosA,_Condition1_PosN,_Condition1_RRAe,_Condition1_RRAn,_Condition1_RRNe,_Condition1_RRNn,_BldgType_1Fam,_BldgType_2fmCon,_BldgType_Duplex,_BldgType_Twnhs,_BldgType_TwnhsE,_HouseStyle_1.5Fin,_HouseStyle_1.5Unf,_HouseStyle_1Story,_HouseStyle_2.5Unf,_HouseStyle_2Story,_HouseStyle_SFoyer,_HouseStyle_SLvl,_RoofStyle_Flat,_RoofStyle_Gable,_RoofStyle_Gambrel,_RoofStyle_Hip,_RoofStyle_Mansard,_RoofStyle_Shed,_Exterior1st_AsbShng,_Exterior1st_AsphShn,_Exterior1st_BrkComm,_Exterior1st_BrkFace,_Exterior1st_CBlock,_Exterior1st_CemntBd,_Exterior1st_HdBoard,_Exterior1st_MetalSd,_Exterior1st_Plywood,_Exterior1st_Stucco,_Exterior1st_VinylSd,_Exterior1st_Wd Sdng,_Exterior1st_WdShing,_Exterior2nd_AsbShng,_Exterior2nd_AsphShn,_Exterior2nd_Brk Cmn,_Exterior2nd_BrkFace,_Exterior2nd_CBlock,_Exterior2nd_CmentBd,_Exterior2nd_HdBoard,_Exterior2nd_ImStucc,_Exterior2nd_MetalSd,_Exterior2nd_Plywood,_Exterior2nd_Stone,_Exterior2nd_Stucco,_Exterior2nd_VinylSd,_Exterior2nd_Wd Sdng,_Exterior2nd_Wd Shng,_Foundation_BrkTil,_Foundation_CBlock,_Foundation_PConc,_Foundation_Slab,_Foundation_Stone,_Foundation_Wood,_SaleType_COD,_SaleType_CWD,_SaleType_Con,_SaleType_ConLD,_SaleType_ConLI,_SaleType_ConLw,_SaleType_New,_SaleType_Oth,_SaleType_WD,_SaleCondition_Abnorml,_SaleCondition_AdjLand,_SaleCondition_Alloca,_SaleCondition_Family,_SaleCondition_Normal,_SaleCondition_Partial,_MasVnrType_BrkCmn,_MasVnrType_BrkFace,_MasVnrType_None,_MasVnrType_Stone,_LotShape_IR1,_LotShape_IR2,_LotShape_IR3,_LotShape_Reg,_LandContour_Bnk,_LandContour_HLS,_LandContour_Low,_LandContour_Lvl,_LandSlope_Gtl,_LandSlope_Mod,_LandSlope_Sev,_Electrical_FuseA,_Electrical_FuseF,_Electrical_FuseP,_Electrical_SBrkr,

In [204]:
X = train.ix[:,:-2]
y = train.ix[:,-2:-1]

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=42)

In [205]:
'''model = ensemble.ExtraTreesRegressor(n_estimators=250,
                              random_state=42)

model.fit(X_train, y_train.values.reshape(-1))
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the model
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X_train.shape[1]), indices)
plt.xlim([-1, X_train.shape[1]])
plt.show()'''

'model = ensemble.ExtraTreesRegressor(n_estimators=250,\n                              random_state=42)\n\nmodel.fit(X_train, y_train.values.reshape(-1))\nimportances = model.feature_importances_\nstd = np.std([tree.feature_importances_ for tree in model.estimators_],\n             axis=0)\nindices = np.argsort(importances)[::-1]\n\n# Print the feature ranking\nprint("Feature ranking:")\n\nfor f in range(X.shape[1]):\n    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))\n\n# Plot the feature importances of the model\nplt.figure()\nplt.title("Feature importances")\nplt.bar(range(X_train.shape[1]), importances[indices],\n       color="r", yerr=std[indices], align="center")\nplt.xticks(range(X_train.shape[1]), indices)\nplt.xlim([-1, X_train.shape[1]])\nplt.show()'

In [206]:
model = linear_model.Ridge(max_iter=100000, alpha=10)

model.fit(X_train, y_train)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=100000,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [207]:
flename = 'models/%s-%s.pkl' % (model_name, timestamp)

joblib.dump(model, flename)

log = '%s,%s\n' % (flename, model)

fle = open('model_log.csv', 'a')
fle.write(log)
fle.flush()
fle.close()

print flename

models/ridge_minimal-2017-01-17-23-33-45.pkl


In [208]:
y_pred = model.predict(X_test)

In [209]:
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
rmse

0.11928593392511261

In [210]:
# Submission
X_sub = test.ix[:,:-1]
y_sub = np.expm1(model.predict(X_sub))

submission = pd.DataFrame()
submission['Id'] = test.Id
submission['SalePrice'] = y_sub

flename = 'submissions/submission-%s-%s.csv' % (model_name, timestamp)
submission.to_csv(flename, index=False)
print flename

submissions/submission-ridge_minimal-2017-01-17-23-33-45.csv
